# AutoMPC Demo

Welcome!  This notebook demonstrates the core features of AutoMPC.

AutoMPC is designed to simplify the process of creating a controller for a robot system with unknown dynamics.
A standard approach to solving this problem is 1) use a System ID algorithm to produce a model of the system dynamics
from data, 2) design an objective which captures the task you want to solve, 3) use an optimization algorithm to solve for a control sequence with respect  to the model and objective function.  AutoMPC provides a toolbox of algorithms for  all three steps, and automates the process of hyperparameter selection for each component.

In this notebook, we will use the cart-pole swing-up task as an example.  Although we know the ground truth dynamics for the cart-pole system, we will demonstrate how to work with an unknown dynamical system by using a multi-layer perceptron (MLP) to learn the dynamics from data.  We will design our model predictive controller using a standard quadratic objective function and an iterative LQR optimizer.

## Set-Up

First, we will import the autompc library and other necessary dependancies.

In [3]:
import cyipopt

In [4]:
import os
os.chdir('/home/randomgraph/baoyul2/meta/autompc')
import autompc as ampc
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

Loading AutoMPC...


ImportError: cannot import name 'pinv2' from 'scipy.linalg' (/home/randomgraph/anaconda3/envs/meta/lib/python3.9/site-packages/scipy/linalg/__init__.py)

The benchmarks module provides some tools for quickly building example problems.  For a list of available benchmarks, see [here](https://autompc.readthedocs.io/en/latest/source/benchmarks.html#available-benchmarks).  Here we will import the cart-pole benchmark

In [ ]:
from autompc.benchmarks import CartpoleSwingupV2Benchmark

benchmark = CartpoleSwingupV2Benchmark()

The `system` object defines the observation and contorl dimensions of the cartpole, while the `task` defines the task we want to solve.  Here, we will just get these from the benchmark, but for more details on systems see example [1. Basics](https://github.com/williamedwards/autompc/tree/main/examples) and for more details on tasks see example [3. Controllers and Tasks](https://github.com/williamedwards/autompc/tree/main/examples).

In [ ]:
# Get system and task specification
system = benchmark.system
task   = benchmark.task

We also need a dataset of trajectories sampled from the system to use for system identification and tuning.  On a real system, we would collect these from the robot, but here we'll use another method from our benchmark to generate the dataset.  The default option is to generate trajectories using uniform random controls.

In [ ]:
# Generate benchmark dataset
trajs = benchmark.gen_trajs(seed=100, n_trajs=100, traj_len=200)

The benchmark also provides capabilities to visualize trajectories.  We can use this to visualize one of the trajectories in our training set.  Since the dataset was generated using uniform random controls, we see that this trajectory clearly does not accomplish the cart-pole swing-up task.  (Note: the animation may take a minute to generate).

In [ ]:
from IPython.display import HTML

# fig = plt.figure()
# ax = fig.gca()
# anim = benchmark.visualize(fig, ax, trajs[1])
# HTML(anim.to_html5_video())

Next, we need to create an MPC to be tuned.  Here, we will use the most general form, AutoSelectController, which allows AutoMPC to automatically select between all of its system ID, optimization, and OCP generation algorithms.

In [ ]:
from autompc import AutoSelectController
controller = AutoSelectController(system)

We can view the tunable hyperparameters of the controller.  Since AutoSelectController chooses between all possible models and optimizers, the hyperparameter space is quite large!

In [ ]:
controller.set_ocp(benchmark.task.get_ocp())
print(controller.get_config_space())

Forbidding model MLP to be used with LQR due to property is_linear
Forbidding model SINDy to be used with LQR due to property is_linear
Checking compatibility with OCP
Requiring cost transformer for ocp to be used with IterativeLQR due to property is_twice_diff
Requiring cost transformer for ocp to be used with LQR due to property is_quad
Configuration space object:
  Hyperparameters:
    ARX:history, Type: UniformInteger, Range: [1, 10], Default: 4
    DirectTranscription:horizon, Type: UniformInteger, Range: [1, 30], Default: 10
    GaussRegTransformer:reg_weight, Type: UniformFloat, Range: [0.001, 10000.0], Default: 1.0, on log-scale
    IterativeLQR:frequency, Type: UniformInteger, Range: [1, 5], Default: 1
    IterativeLQR:horizon, Type: UniformInteger, Range: [5, 25], Default: 20
    IterativeLQR:max_iter, Type: UniformInteger, Range: [10, 50], Default: 20
    Koopman:lasso_alpha, Type: UniformFloat, Range: [1e-10, 100.0], Default: 1.0, on log-scale
    Koopman:method, Type: Cate

## Tuning

Once we have initialized the pipeline and its factories, we can set up a tuner to automatically search over the pipeline's configuration space.  Since we assume we don't have access to the ground truth dynamics, we train a surrogate dynamics model, which is used as a simulator to evaluate configurations.

Here we pass an `MLP` instance to the tuner to be used to train the surrogate dynamics model.  The `surrogate_split` controls what proportion of the data will be used for training the surrogate dynamics model vs training the system ID model.

This is an example of full pipeline which searches the configuration space of all pipeline components simultaneously.  (NOTE: This takes quite a while to run.  Skip this cell and run the next one instead to load a cached result).

In [ ]:
from autompc.tuning import ControlTuner
from autompc.sysid import MLP

tuner = ControlTuner(surrogate=MLP(system), surrogate_split=0.5, surrogate_tune_horizon=5)

We can now run the tuning porcess.  In addition to the pipeline, we pass the task and the trajectory dataset.  The task specification is used to evaluate the trajectories simulated with the surrogate dynamics model.  AutoMPC uses the `smac3` package for Bayesian optimization to search the configuration space.

Here, we pass the ground truth dynamics to the tuner so that we can measure the true performance over time.  However, we would not have this for a real robot, so it's not used for selecting the configuration.

**Note:** The tuning process can take 5 or more hours to run depending on the hardware available.  To load a cached result instead of running this yourself, skip this cell and run the next one instead.

In [ ]:
tuned_controller, tune_result = tuner.run(controller, task, trajs, n_iters=100, rng=np.random.default_rng(100), 
                                   truedyn=benchmark.dynamics)
                                   #restore_dir="autompc-output_2022-02-10T17:10:43")

INFO:smac.utils.io.cmd_reader.CMDReader:Output to smac3-output_2022-08-14_15:40:28_912830
INFO:smac.facade.smac_ac_facade.SMAC4AC:Optimizing a deterministic scenario for quality without a tuner timeout - will make SMAC deterministic and only evaluate one configuration per iteration!
INFO:smac.initial_design.default_configuration_design.DefaultConfiguration:Running initial design for 1 configurations
INFO:smac.optimizer.smbo.SMBO:Running initial design
INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent


------------------------------------------------------------------
Beginning surrogate tuning with model class MLP
------------------------------------------------------------------
100%|██████████| 200/200 [00:26<00:00,  7.51it/s]

INFO:smac.intensification.intensification.Intensifier:First run, no incumbent provided; challenger is assumed to be the incumbent
INFO:smac.intensification.intensification.Intensifier:Updated estimated cost of incumbent on 1 runs: 0.0814



100%|██████████| 200/200 [00:35<00:00,  5.62it/s]

INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 292.962184 sec, available: 0.000010 sec)


...


100%|██████████| 200/200 [00:47<00:00,  4.19it/s]

INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 348.243001 sec, available: 0.000010 sec)



100%|██████████| 200/200 [00:16<00:00, 11.96it/s]

INFO:smac.intensification.intensification.Intensifier:Wallclock time limit for intensification reached (used: 121.329397 sec, available: 0.000010 sec)



 54%|█████▍    | 108/200 [00:16<00:14,  6.51it/s]

Run the below cell to load a cached tune result

In [ ]:
#import os, pickle
#current_folder = globals()['_dh'][0]
#fn = os.path.join(current_folder, "../assets/cached_tunes/cartpole_tune_result.pkl")
#with open(fn, "rb") as f:
#    tune_result = pickle.load(f)
#inc_cfg = tune_result.inc_cfg
#controller, cost, model = pipeline(inc_cfg, task, trajs)

We can then plot the performance of the best controller found so far over the iterations of the tuning process.  We plot both the performance as evaluated with the surrogate dynamics and the true dynamics performance.  Lower scores are better, so after 100 iterations, both the surrogate and true performance converge to a good solution.

In [ ]:
from autompc.graphs import TuningCurveGraph
import matplotlib.pyplot as plt

graph = TuningCurveGraph()

fig = plt.figure()      
ax = fig.gca()
graph(ax, tune_result)
ax.set_title("Cart-Pole Tuning Curve")
plt.show()

We can also view the configurations selected by the tuner.

In [ ]:
tune_result.inc_cfg

Now that our tuning process has given us a controller, we can run it to simulate a trajectory.  For more information on how to work with controllers, see example [3. Controllesr .

In [ ]:
traj = ampc.simulate(tuned_controller, init_obs=benchmark.task.get_init_obs(), max_steps=200, dynamics=benchmark.dynamics)

In [ ]:
fig = plt.figure()
ax = fig.gca()
anim = benchmark.visualize(fig, ax, traj)
HTML(anim.to_html5_video())

In [ ]:
import pickle
with open("../assets/cached_tunes/cartpole_v2_result_01.pkl", "wb") as f:
    pickle.dump(tune_result, f)